In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from scipy.linalg import sqrtm
from copy import deepcopy
from datetime import datetime
import time

In [2]:
## load data
train = pd.read_csv('products.csv')
test = pd.read_csv('sampleSubmission.csv')

C:\Anaconda3\envs\aind-dog-copy\lib\site-packages\IPython\core\interactiveshell.py:2717: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
## fill missing values
#products['customerID'].fillna('BBID_0000', inplace=True)
train['promotion_description'].fillna('no_promo', inplace=True)
train['Gender'].fillna('no_gender', inplace=True)
train['State'].fillna('no_state', inplace=True)
train['PinCode'].fillna(-1, inplace=True)
train['DOB'].fillna("1", inplace=True)

In [4]:
#handling missing data for product_code
train = train[np.isfinite(train['product_code'])]

In [5]:
state_dict = {'MADHY PRADESH':'MADHYA PRADESH', 'TAMILNADU':'TAMIL NADU', 'MADHYA  PRADESH':'MADHYA PRADESH', 'HARAYANA':'HARYANA',
             'Jharkhand':'JHARKHAND','Tamilnadu':'TAMIL NADU','Tamil Nadu':'TAMIL NADU','Madhya Pradesh':'MADHYA PRADESH',
             'REST OF WEST BENGAL':'WEST BENGAL', 'west bengal':'WEST BENGAL','Uttar Pradesh':'UTTAR PRADESH', 'Delhi':'DELHI',
             'Bhopal':'BHOPAL','CHHATISGARH':'CHHATTISGARH','CHATTISGARH':'CHHATTISGARH', 'jharkhand':'JHARKHAND','Chandigarh':'CHANDIGARH',
             'UTTAR PRADESH WEST': 'UTTAR PRADESH','ODISHA':'ORISSA','MAHARASTRA':'MAHARASHTRA','madhya pradesh':'MADHYA PRADESH',
             'KARNATAK':'KARNATAKA','JAMMU and KASHMIR':'JAMMU AND KASHMIR','JAMMU KASHMIR':'JAMMU AND KASHMIR','Rajasthan':'RAJASTHAN',
             'east singhbhum':'JHARKHAND', 'ORRISA':'ORISSA','Andhra Pradesh':'ANDHRA PRADESH', 'UTTARANCHAL':'UTTARAKHAND',
             'Uttar pradesh':'UTTAR PRADESH','Maharashtra':'MAHARASHTRA','MP':'MADHYA PRADESH', 'UTTAR PRADESH EAST':'UTTAR PRADESH',
             'Punjab':'PUNJAB','maharashtra':'MAHARASHTRA','Karnataka':'KARNATAKA','M.P.':'MADHYA PRADESH','DAMAN':'DAMAN AND DIU',
             'HUBLI':'KARNATAKA','Tamil nadu':'TAMIL NADU','GUJRAT':'GUJARAT', 'Mp':'MADHYA PRADESH','Madhya pradesh':'MADHYA PRADESH',
             'West Bengal':'WEST BENGAL','Gujarat':'GUJARAT','UP':'UTTAR PRADESH','Chennai':'CHENNAI', 'm.p.':'MADHYA PRADESH',
             'kerala':'KERALA'}

train.replace({"State": state_dict}, inplace=True)

In [6]:
cnt_srs = train['product_code'].astype('int64').value_counts().reset_index().head(100)
#cnt_srs = products['product_code'].value_counts().reset_index()
cnt_srs.columns = ['product_code', 'frequency_count']
cnt_srs

,product_code,frequency_count
0,300776411,60994
1,300776410,58437
2,108037568,51338
3,300785148,41500
4,108100382,35259
5,300785150,34794
6,300840018,31149
7,108100362,27963
8,108005676,27727
9,300111517,25360


In [7]:
prod_index = []
for i in cnt_srs['product_code']:
    prod_index.append(i)

In [8]:
product_joined = pd.merge(train, cnt_srs, on='product_code', how='inner')
products_less = product_joined[['customerID', 'Gender', 'State', 'transactionDate', 'product_code']].reset_index()
products_less.drop('index', axis=1, inplace=True)
products_to_join = pd.DataFrame(train['customerID']).reset_index()
products_to_join.drop('index', axis=1, inplace=True)

In [9]:
# Join the dummy variables to the main dataframe
train = pd.merge(products_to_join, products_less.drop_duplicates(subset=['customerID'], keep='last'), on='customerID', how='left')
train.head()

,customerID,Gender,State,transactionDate,product_code
0,BBID_2041,male,no_state,2016-03-20,1.081e+08
1,BBID_2041,male,no_state,2016-03-20,1.081e+08
2,BBID_2041,male,no_state,2016-03-20,1.081e+08
3,BBID_2041,male,no_state,2016-03-20,1.081e+08
4,BBID_2041,male,no_state,2016-03-20,1.081e+08


In [10]:
train.dropna(inplace=True)

In [11]:
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7821602 entries, 0 to 7981241
Data columns (total 5 columns):
customerID         object
Gender             object
State              object
transactionDate    object
product_code       object
dtypes: object(5)
memory usage: 358.0+ MB


In [12]:
train['product_code'] = train['product_code'].astype(int)

In [13]:
prod_dict = {'_'}

train = pd.get_dummies(train, columns=['product_code'], prefix=prod_dict)
train.columns=train.columns.str.replace('_','')
train.reset_index(drop=True).head()

,customerID,Gender,State,transactionDate,100105505,108000537,108000568,108000707,108001125,108001127,...,300808923,300840018,300942704,300974316,300974360,500096181,1000010411,1000058092,1000336252,1000609658
0,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_2041,male,no_state,2016-03-20,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
df_train = train.copy()

In [15]:
df_june = df_train[df_train['transactionDate'] == '2017-06-28']

In [16]:
df_june.reset_index(drop=True)

,customerID,Gender,State,transactionDate,100105505,108000537,108000568,108000707,108001125,108001127,...,300808923,300840018,300942704,300974316,300974360,500096181,1000010411,1000058092,1000336252,1000609658
0,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
6,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
7,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
8,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
9,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [17]:
product_col = ['100105505', '108000537', '108000568', '108000707',
       '108001125', '108001127', '108003448', '108003451',
       '108004423', '108004880', '108004977', '108005676',
       '108005681', '108008866', '108010225', '108015124',
       '108015133', '108017242', '108017245', '108018729',
       '108018730', '108018925', '108020429', '108030063',
       '108030113', '108032048', '108037499', '108037504',
       '108037568', '108043278', '108100137', '108100143',
       '108100145', '108100183', '108100193', '108100243',
       '108100253', '108100267', '108100278', '108100288',
       '108100290', '108100294', '108100296', '108100297',
       '108100298', '108100302', '108100306', '108100308',
       '108100319', '108100328', '108100330', '108100331',
       '108100334', '108100335', '108100340', '108100352',
       '108100361', '108100362', '108100382', '300028156',
       '300070102', '300088564', '300111517', '300143077',
       '300144423', '300150379', '300157247', '300177372',
       '300317078', '300326493', '300347917', '300361324',
       '300374464', '300481673', '300481740', '300481741',
       '300497842', '300528552', '300616595', '300647581',
       '300676075', '300680079', '300685217', '300776409',
       '300776410', '300776411', '300781593', '300785147',
       '300785148', '300785150', '300808923', '300840018',
       '300942704', '300974316', '300974360', '500096181',
       '1000010411', '1000058092', '1000336252', '1000609658']

In [18]:
month = 2

df_train_5 = df_train.loc[df_train['transactionDate']=='2017-05-28', product_col+['customerID']]
df_train_4 = df_train.loc[df_train['transactionDate']=='2017-04-28', product_col+['customerID']]
#df_train_3 = df_train.loc[df_train['transactionDate']=='2017-03-28', product_col+['customerID']]
#df_train_2 = df_train.loc[df_train['transactionDate']=='2017-02-28', product_col+['customerID']]
#df_train_1 = df_train.loc[df_train['transactionDate']=='2017-01-28', product_col+['customerID']]

In [19]:
df_train_5.head()

,100105505,108000537,108000568,108000707,108001125,108001127,108003448,108003451,108004423,108004880,...,300840018,300942704,300974316,300974360,500096181,1000010411,1000058092,1000336252,1000609658,customerID
8554,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BBID_204267
8555,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BBID_204267
8556,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BBID_204267
8557,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BBID_204267
8558,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,BBID_204267


In [20]:
dfm = pd.merge(df_june,df_train_5, how='left', on=['customerID'], suffixes=('', '_5'))
dfm = pd.merge(dfm,df_train_4, how='left', on=['customerID'], suffixes=('', '_4'))
#dfm = pd.merge(dfm,df_train_3, how='left', on=['customerID'], suffixes=('', '_3'))
#dfm = pd.merge(dfm,df_train_2, how='left', on=['customerID'], suffixes=('', '_2'))
#dfm = pd.merge(dfm,df_train_1, how='left', on=['customerID'], suffixes=('', '_1'))

In [21]:
dfm.head()

,customerID,Gender,State,transactionDate,100105505,108000537,108000568,108000707,108001125,108001127,...,300808923_4,300840018_4,300942704_4,300974316_4,300974360_4,500096181_4,1000010411_4,1000058092_4,1000336252_4,1000609658_4
0,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
all_product_col = [col for col in dfm.columns if '' in col]

for col in all_product_col:
    dfm[col].fillna(0, inplace=True)

In [23]:
dfm.head()

,customerID,Gender,State,transactionDate,100105505,108000537,108000568,108000707,108001125,108001127,...,300808923_4,300840018_4,300942704_4,300974316_4,300974360_4,500096181_4,1000010411_4,1000058092_4,1000336252_4,1000609658_4
0,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [24]:
for col in product_col:
    dfm[col+'_5'] =dfm[col]
    
dfm.drop(product_col, axis=1, inplace=True)

In [25]:
dfm.head()

,customerID,Gender,State,transactionDate,100105505_5,108000537_5,108000568_5,108000707_5,108001125_5,108001127_5,...,300808923_4,300840018_4,300942704_4,300974316_4,300974360_4,500096181_4,1000010411_4,1000058092_4,1000336252_4,1000609658_4
0,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,BBID_204975,male,MADHYA PRADESH,2017-06-28,0,0,0,0,0,0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [26]:
dfm.drop('transactionDate', axis=1, inplace=True)

In [27]:
dfm.to_csv('cleaned_data/TestSet_withpast3_new2_woDecimal.csv', index=False)